In [1]:
# Load dataset

import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

## 
data = gutenberg.raw('shakespeare-hamlet.txt')
# save to a file
with open('hamlet.txt' ,'w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to /Users/roopa/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [5]:
## data preprocessing

import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## load the dataset
with open('hamlet.txt','r') as file:
    text = file.read().lower()

## tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words



4818

In [14]:
## create input sequences

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i]
        input_sequences.append(n_gram_sequence)
input_sequences

[[1],
 [1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1180],
 [1180, 1889],
 [1180, 1889, 1890],
 [57],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [407],
 [407, 1182],
 [408],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [18],
 [451],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [408],
 [451],
 [408],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [451],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [408],
 [408, 20],
 [408, 20, 16],
 [408, 20, 16, 1895],
 [408, 20, 16, 1895, 114],
 [408, 20, 16, 1895, 114, 379],
 [408, 20, 16, 1895, 114, 379, 71],
 [408, 20, 16, 1895

In [15]:
## Pad Sequences

max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

13

In [17]:
from tensorflow.keras.preprocessing import sequence
input_sequences = sequence.pad_sequences(input_sequences , maxlen= max_sequence_len , padding ='pre')
input_sequences

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]], dtype=int32)

In [19]:
## create predictors and label

import tensorflow as tf
x , y = input_sequences[:,:-1] , input_sequences[:,-1]
x , y

(array([[   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    1],
        [   0,    0,    0, ...,    0,    1,  687],
        ...,
        [   0,    0,    0, ...,    1,  687,    4],
        [   0,    0,    0, ...,  687,    4,   45],
        [   0,    0,    0, ...,    4,   45, 1047]], dtype=int32),
 array([   1,  687,    4, ...,   45, 1047,    4], dtype=int32))

In [20]:
y = tf.keras.utils.to_categorical(y , num_classes= total_words)
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
# split the data into training and testing set
x_train , x_test ,y_train, y_test = train_test_split(x,y , test_size= 0.2)


In [24]:
## Train our LSTM  RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding , LSTM , Dense , Dropout


# define the model
model = Sequential()
model.add(Embedding(total_words ,100, input_length = max_sequence_len))
model.add(LSTM(150, return_sequences = True))
model.add(Dropout (0.2))
model.add(LSTM(100))
model.add(Dense(total_words , activation = 'softmax'))

##compile the model
model.compile(loss = "categorical_crossentropy" , optimizer = 'adam' , metrics = ['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define early stopping

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_loss' ,patience = 3, restore_best_weights=True)



In [25]:
## Train the model

history = model.fit(x_train , y_train , epochs= 50 , validation_data= (x_test , y_test) , verbose=1 ,callbacks = [early_stopping])


Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.0344 - loss: 6.8947 - val_accuracy: 0.0371 - val_loss: 6.4841
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0412 - loss: 6.1571 - val_accuracy: 0.0567 - val_loss: 6.5297
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.0556 - loss: 5.9734 - val_accuracy: 0.0573 - val_loss: 6.5948
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0584 - loss: 5.8493 - val_accuracy: 0.0591 - val_loss: 6.6122
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0652 - loss: 5.7672 - val_accuracy: 0.0622 - val_loss: 6.6524
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0700 - loss: 5.6135 - val_accuracy: 0.0602 - val_loss: 6.6864
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0785 - loss: 5.4977 - val_accuracy: 0.0692 - val_loss: 6.7130
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0861 - loss: 5.3498 - val_acc

In [66]:
## Function to predict the next word

def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    #if len(token_list) >= max_sequence_len:
    token_list = token_list[-(max_sequence_len):]
    token_list = pad_sequences([token_list] ,maxlen=max_sequence_len , padding='pre')
    predicted =model.predict(token_list,verbose=0)
    predict_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predict_word_index:
            return word
        #return None


In [67]:
model.input_shape

(None, 12)

In [68]:
input_text = 'To be or not to be'
max_sequence_len = len(list(input_text))+1
next_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
next_word

'considered'

In [69]:
#save the model
model.save("next_word_lstm.h5")
## save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)